In [ ]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
# for one hot encoding.
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import vgg16 as vgg
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [ ]:
# loading the dataset.
(xtrain, ytrain), (xtest, ytest) = keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 14s 0us/step


In [ ]:
## limitting number of images for saving resources
xtrain=xtrain[:5000]
ytrain=ytrain[:5000]
xtest=xtest[:1000]
ytest=ytest[:1000]

In [ ]:
## converting to input format of AlexNet.
xtrain=np.array([cv2.resize(i, (224,224)) for i in xtrain])  ## cv2 channel issue not addressed?
xtest=np.array([cv2.resize(i, (224,224)) for i in xtest])

In [ ]:
## let's rescale all pixel values to [0,1] range.

# convert from integers to floats
# normalize to range 0-1
xtrain = xtrain.astype('float32')/255
xtest = xtest.astype('float32')/255

In [ ]:
ytrain = to_categorical(ytrain, 10) # 10 indicates total number of unique labels.
ytest = to_categorical(ytest, 10)

In [ ]:
loaded_model=vgg.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
last_layer=loaded_model.get_layer('block3_pool').output

In [ ]:
x=GlobalAveragePooling2D()(last_layer)
# x=BatchNormalization(x)
x=Dense(256, activation='relu')(x)
x=Dense(256, activation='relu')(x)
x=Dropout(0.5)(x)
final=Dense(10, activation='softmax')(x)  # 10 is the total number of classes.

## constructing our finalized model.
model=Model(loaded_model.input, final)


In [ ]:
# other are frozen
for layer in loaded_model.layers:  
  layer.trainable=False

In [ ]:
# ## compile model.
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# prepare data augmentation.
train_datagen=ImageDataGenerator(rescale=1/255, rotation_range=20)
train_datagen.fit(xtrain)
train_generator=train_datagen.flow(xtrain, ytrain, batch_size=32)

In [ ]:
# prepare data augmentation.
val_datagen=ImageDataGenerator(rescale=1/255, rotation_range=20)
val_datagen.fit(xtest)
val_generator=val_datagen.flow(xtest, ytest, batch_size=32)

In [ ]:
train_steps=xtrain.shape[0]//32  ## 32 is batch size
val_steps=xtest.shape[0]//32

history=model.fit_generator(
    train_generator, steps_per_epoch=train_steps, 
    validation_data=val_generator, validation_steps=val_steps,
    epochs=10, verbose=1
    )



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
156/156 [==============================] - 78s 419ms/step - loss: 2.6456 - accuracy: 0.1012 - val_loss: 2.3023 - val_accuracy: 0.1069
Epoch 2/10
156/156 [==============================] - 65s 417ms/step - loss: 2.3025 - accuracy: 0.1019 - val_loss: 2.3021 - val_accuracy: 0.1058
Epoch 3/10
156/156 [==============================] - 66s 420ms/step - loss: 2.3023 - accuracy: 0.0984 - val_loss: 2.3020 - val_accuracy: 0.1048
Epoch 4/10
156/156 [==============================] - 65s 418ms/step - loss: 2.3022 - accuracy: 0.1004 - val_loss: 2.3019 - val_accuracy: 0.1058
Epoch 5/10
156/156 [==============================] - 65s 417ms/step - loss: 2.3022 - accuracy: 0.1017 - val_loss: 2.3020 - val_accuracy: 0.1058
Epoch 6/10
156/156 [==============================] - 65s 416ms/step - loss: 2.3021 - accuracy: 0.1019 - val_loss: 2.3020 - val_accuracy: 0.1008
Epoch 7/10
156/156 [==============================] - 65s 415ms/step - loss: 2.3022 - accuracy: 0.0984 - val_loss: 2.3023 - val_ac

In [ ]:
# evaluate model
__, acc = model.evaluate(xtest, ytest, verbose=0)
print("accuracy: ", acc*100)

accuracy:  9.300000220537186


In [ ]:
model.predict(xtest[:1])

array([[1.4838192e-04, 2.5567864e-03, 3.5295663e-03, 3.2458559e-01,
        5.7715300e-04, 3.1639403e-03, 7.9576802e-03, 6.0776019e-01,
        1.2042059e-05, 4.9708571e-02]], dtype=float32)

In [ ]:
# ========================================= THE END =========================================